## Camera Calibration

Relevant information can be found in the OpenCV [OpenCV](https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html) documentation

In [6]:
# First create a python virtual environment

# Install this packages
#!pip install numpy
#!pip install opencv-python

Obtain objpoints from the images

In [7]:
import numpy as np
import cv2 as cv
import glob
import os

# termination criteria for refining corner points
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0), ....,(6,5,0)
objp = np.zeros((6*7, 3), np.float32)
objp[:, :2] = np.mgrid[0:7, 0:6].T.reshape(-1, 2)

# Arrays to store object points and image points from all the images.
objpoints = []  # 3d points in real world space
imgpoints = []  # 2d points in image plane.

# Get all image file paths from the folder
images = glob.glob('Calibration_Images/Images/*.jpg')

# Create directory for saving results if not exists
output_dir = "Calibration_Images/Corner_Results"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for idx, fname in enumerate(images):
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv.findChessboardCorners(gray, (7, 6), None)

    # If corners are found
    if ret:
        objpoints.append(objp)

        # Refine corner locations
        corners2 = cv.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        cv.drawChessboardCorners(img, (7, 6), corners2, ret)

        # Save the result with corners drawn
        result_filename = os.path.join(output_dir, f"corner_result_{idx+1}.jpg")
        cv.imwrite(result_filename, img)

        # Show the image (optional, might not work properly in non-GUI environments)
        cv.imshow('img', img)
        cv.waitKey(500)

cv.destroyAllWindows()

Obtain calibration camera results

In [9]:
# Calibrate the camera
ret, camera_matrix, distortion_coeffs, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# Print calibration results
print("Camera matrix:\n", camera_matrix)
print("Distortion coefficients:\n", distortion_coeffs)
print("Rotation Vectors:\n", rvecs)
print("Translation Vectors:\n", tvecs)

Camera matrix:
 [[938.23561468   0.         621.98907581]
 [  0.         926.58473294 355.6305686 ]
 [  0.           0.           1.        ]]
Distortion coefficients:
 [[ 2.13378982e-02 -3.46887436e-01 -1.37717511e-04 -1.02818189e-02
   6.29159483e-01]]
Rotation Vectors:
 (array([[ 0.7863924 ],
       [ 0.03652729],
       [-0.00468984]]), array([[-0.68968145],
       [-0.29945532],
       [-0.64320047]]), array([[-0.83931219],
       [-0.43620673],
       [-0.41598355]]), array([[-0.8022483 ],
       [ 0.01227122],
       [ 0.6309271 ]]))
Translation Vectors:
 (array([[-3.45560008],
       [-1.43728664],
       [17.64759332]]), array([[-8.56145243],
       [ 0.37347525],
       [18.52851413]]), array([[-6.56573282],
       [-0.12017605],
       [17.13930775]]), array([[-0.24420479],
       [-3.1445083 ],
       [19.46807773]]))


Undistort an image

In [13]:
# Undistort one of the test images
img = cv.imread(images[5])
h,  w = img.shape[:2]
new_camera_matrix, roi = cv.getOptimalNewCameraMatrix(camera_matrix, distortion_coeffs, (w,h), 1, (w,h))

# Undistort
undistorted_img = cv.undistort(img, camera_matrix, distortion_coeffs, None, new_camera_matrix)

# Crop the image if necessary
x, y, w, h = roi
undistorted_img = undistorted_img[y:y + h, x:x + w]

# Save and display the undistorted image
cv.imwrite('Calibration_Images/undistorted_result.jpg', undistorted_img)

True